In [4]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import requests
import argparse
import sys

# -*- coding: utf-8 -*-


In [5]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [6]:
def embassies():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    resultado = response.json()
    embassies = pd.json_normalize(resultado['@graph'])
    embassies["Type of place"]= "Embajadas y Consulados"
    embassies = embassies.rename(columns={"title": "Place of interest", "address.street-address": "Place address", "location.longitude": "Longitude_start", "location.latitude": "Latitude_start"}, errors="raise")
    embassies_def = embassies[["Place of interest","Type of place","Place address","Longitude_start","Latitude_start"]]
    embassies_def['Latatitude_start'] = pd.to_numeric(embassies_def['Latitude_start'],errors = 'coerce')
    embassies_def['Longitude_start'] = pd.to_numeric(embassies_def['Longitude_start'],errors = 'coerce')
    return embassies_def

def stations():
    stations = pd.read_json("data/bicimad.json", orient='records')
    geometry_coordinates = stations["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    stations = pd.concat([stations, geometry_coordinates], axis=1)
    stations_def = stations.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitude_finish", "LATITUD": "Latitude_finish"}, errors="raise")
    stations_def = stations_def[["BiciMAD station","Station location","Longitude_finish","Latitude_finish"]]
    stations_def['Latitude_finish'] = pd.to_numeric(stations_def['Latitude_finish'],errors = 'coerce')
    stations_def['Longitude_finish'] = pd.to_numeric(stations_def['Longitude_finish'],errors = 'coerce')
    return stations_def


In [7]:
def mercator_1():
    embassies_def["mercator_start"] = embassies_def.apply(lambda x: to_mercator(x['Latitude_start'],x['Longitude_start']),axis=1)
    return embassies_def
def mercator_2():
    stations_def["mercator_finish"] = stations_def.apply(lambda x: to_mercator(x['Latitude_finish'],x['Longitude_finish']),axis=1)
    return stations_def

In [8]:
def merge():
    df_resultado = pd.merge(embassies_def,stations_def, how="cross")
    return df_resultado

In [9]:
def apply_distance():
    df_resultado["Distance"] = df_resultado.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)
    return df_resultado


In [10]:
def minimum():
    df_min = df_resultado[df_resultado["Place of interest"] == input('Enter a place of interest: ')]
    minimum = df_min[df_min['Distance'] == df_min['Distance'].min()]
    minimum_mod = minimum[["Place of interest","Type of place","Place address","BiciMAD station","Station location","Distance"]]
    return minimum_mod


In [11]:
#def all_places():
    #df_minimum = pd.DataFrame(columns=["Place of interest","Type of place","Place address","BiciMAD station","Station location"])
    #for place in monuments_clean["Place of interest"]:
        #df_filter2 = df_sumary[df_sumary["Place of interest"] == place]
        #df_places =  df_filter2[df_filter2['Distance'] == df_filter2['Distance'].min()]
        #df_places_clear = df_places[["Place of interest","Type of place","Place address","BiciMAD station","Station location"]]
        #df_minimum = df_minimum.append(df_places_clear)
    #return df_minimum

    

In [12]:
def all_places():  
    return df_resultado.sort_values(by = "Distance", ascending = True).groupby('Place of interest')['Type of place','Place address','BiciMAD station', 'Station location','Distance'].nth(0).drop(["Distance"], axis = "columns") 



In [13]:
stations_def = stations()
embassies_def = embassies()
embassies_def = mercator_1()
stations_def = mercator_2()
df_resultado = merge()
df_resultado = apply_distance()
all_places()

/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()
/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,
Consulado de Argentina,Embajadas y Consulados,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,Alonso Martínez,Plaza de Alonso Martínez nº 5
Consulado de Belice,Embajadas y Consulados,CALLE TALAVERA 9,Serrano 210,Calle Serrano nº 210
Consulado de Bolivia,Embajadas y Consulados,CALLE AVIADOR LINDBERGH 3,María Francisca 1,Calle María Francisca nº 1
Consulado de Brasil,Embajadas y Consulados,CALLE GOYA 5 y 7 pasaje,Plaza de Colón,Calle Goya nº 1
Consulado de Bélgica,Embajadas y Consulados,PASEO CASTELLANA 18 PLANTA 6 Izquierda,Castellana,Paseo de la Castellana nº 4
...,...,...,...,...
Embajada de Vietnam,Embajadas y Consulados,AVENIDA ALFONSO XIII 54,Corazón de María,Calle Santa Hortensia nº 31
Embajada de Yemen,Embajadas y Consulados,PASEO CASTELLANA 117 PLANTA 8 D,Sor Ángela de la Cruz,Calle Sor Ángela de la Cruz nº 2
Embajada de la República Popular Democrática de Corea,Embajadas y Consulados,CALLE DARIO APARICIO 43,Facultad Derecho,Avenida Complutense nº 23


In [14]:
minimum()

Enter a place of interest: Embajada del Nepal


,Place of interest,Type of place,Place address,BiciMAD station,Station location,Distance
42663,Embajada del Nepal,Embajadas y Consulados,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,Paseo de la Castellana - Glorieta de Emilio Ca...,Paseo de la Castellana nº 43,159.333962


In [3]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.
